# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Описание данных

**Признаки**

- *DateCrawled* — дата скачивания анкеты из базы
- *VehicleType* — тип автомобильного кузова
- *RegistrationYear* — год регистрации автомобиля
- *Gearbox* — тип коробки передач
- *Power* — мощность (л. с.)
- *Model* — модель автомобиля
- *Kilometer* — пробег (км)
- *RegistrationMonth* — месяц регистрации автомобиля
- *FuelType* — тип топлива
- *Brand* — марка автомобиля
- *NotRepaired* — была машина в ремонте или нет
- *DateCreated* — дата создания анкеты
- *NumberOfPictures* — количество фотографий автомобиля
- *PostalCode* — почтовый индекс владельца анкеты (пользователя)
- *LastSeen* — дата последней активности пользователя

**Целевой признак**
- *Price* — цена (евро)

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span></li></ul></div>

## Подготовка данных

In [22]:
# выгрузим необходимые библиотеки

import pandas as pd

import numpy as np

from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression

import lightgbm as lgb

from sklearn.metrics import mean_squared_error

import time

from sklearn.dummy import DummyRegressor

In [23]:
# прочитаем данные 

def data_reading(path):
    data = pd.read_csv(path)
    print(f'Информация о данных: \n')
    data.info()
    print(f'\nКоличество дубликатов: {data.duplicated().sum()} \n')
    print(f'\nДоля дубликатов: {data.duplicated().mean()}\n')
    print(f'\nКоличество пропусков:\n\
{data.isna().sum()} \n')
    print(f'\nДоля пропусков:\n\
{data.isna().mean()} \n')
    print(f'Первые 5 строк датасета: \n')
    display(data.head(5))
    return data

In [24]:
data = data_reading('/content/autos.csv')

Информация о данных: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268039 entries, 0 to 268038
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DateCrawled        268039 non-null  object 
 1   Price              268039 non-null  int64  
 2   VehicleType        239657 non-null  object 
 3   RegistrationYear   268039 non-null  int64  
 4   Gearbox            253074 non-null  object 
 5   Power              268038 non-null  float64
 6   Model              253033 non-null  object 
 7   Kilometer          268038 non-null  float64
 8   RegistrationMonth  268038 non-null  float64
 9   FuelType           243102 non-null  object 
 10  Brand              268038 non-null  object 
 11  Repaired           214170 non-null  object 
 12  DateCreated        268038 non-null  object 
 13  NumberOfPictures   268038 non-null  float64
 14  PostalCode         268038 non-null  float64
 15  LastSeen           268038 no

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0.0,golf,150000.0,0.0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0.0,70435.0,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190.0,NaN,125000.0,5.0,gasoline,audi,yes,2016-03-24 00:00:00,0.0,66954.0,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163.0,grand,125000.0,8.0,gasoline,jeep,NaN,2016-03-14 00:00:00,0.0,90480.0,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75.0,golf,150000.0,6.0,petrol,volkswagen,no,2016-03-17 00:00:00,0.0,91074.0,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69.0,fabia,90000.0,7.0,gasoline,skoda,no,2016-03-31 00:00:00,0.0,60437.0,2016-04-06 10:17:21


In [25]:
# приведем названия в соответствии с общепринятыми правилами

data.columns = [column.lower() for column in data.columns]
data.rename(columns={'datecrawled':'date_crawled',
                     'registrationmonth':'registration_month',
                     'datecreated':'date_created',
                     'numberofpictures':'number_of_pictures',
                     'postalcode':'postal_code',
                     'lastseen':'last_seen',
                     'vehicletype':'vehicle_type', 
                     'registrationyear': 'registration_year', 
                     'fueltype':'fuel_type', 
                     'repaired':'not_repaired'}, inplace=True)

data

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0.0,golf,150000.0,0.0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0.0,70435.0,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190.0,NaN,125000.0,5.0,gasoline,audi,yes,2016-03-24 00:00:00,0.0,66954.0,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163.0,grand,125000.0,8.0,gasoline,jeep,NaN,2016-03-14 00:00:00,0.0,90480.0,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75.0,golf,150000.0,6.0,petrol,volkswagen,no,2016-03-17 00:00:00,0.0,91074.0,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69.0,fabia,90000.0,7.0,gasoline,skoda,no,2016-03-31 00:00:00,0.0,60437.0,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268034,2016-04-01 16:53:37,800,small,1998,auto,60.0,fiesta,125000.0,2.0,petrol,ford,no,2016-04-01 00:00:00,0.0,6712.0,2016-04-01 16:53:37
268035,2016-03-29 01:55:57,399,wagon,1998,auto,101.0,astra,150000.0,3.0,petrol,opel,yes,2016-03-29 00:00:00,0.0,36304.0,2016-04-05 17:26:08
268036,2016-03-22 16:36:19,899,small,1994,manual,116.0,golf,150000.0,4.0,petrol,volkswagen,yes,2016-03-22 00:00:00,0.0,72459.0,2016-03-22 17:41:52
268037,2016-03-09 13:43:05,0,NaN,2016,manual,60.0,ibiza,150000.0,4.0,petrol,seat,no,2016-03-09 00:00:00,0.0,93059.0,2016-03-17 07:45:57


In [26]:
# тип данных столбцов с датами - *object*. Заменим его на стандартный *datetime*

for column in ['date_crawled', 'date_created', 'last_seen']:
    data[column] = pd.to_datetime(data[column], format='%Y-%m-%d %H:%M:%S')

Проанализируем столбцы с пропусками

In [27]:
# vehicle_type

print(data['vehicle_type'].unique())

# в столбце фигурирует показатель other, которым мы можем заполнить пропуски 

data['vehicle_type'] = data['vehicle_type'].fillna('other')

[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']


In [28]:
# gearbox

print(data['gearbox'].value_counts())

# преобладают авто с ручной каробкой передач, 
# однако конкретно определить все пропуски в данную категорию будет неправильно 
# как и в прошлый раз, заполнним данные значением other

data['gearbox'] = data['gearbox'].fillna('other')


manual    202972
auto       50101
man            1
Name: gearbox, dtype: int64


In [29]:
# model

print(data['model'].unique())

# в столбце фигурирует показатель other, которым мы можем заполнить пропуски

data['model'] = data['model'].fillna('other')


['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuore' 's_max' 'a2' 'galaxy' 'c3' 'viano' 's_klasse'
 '1_reihe' 'avensis' 'roomster' 'sl' 'kaefer' 'santa' 'cooper' 'leon'
 '4

In [30]:
# fuel_type

print(data['fuel_type'].value_counts())

# в столбце фигурирует показатель other, которым мы можем заполнить пропуски

data['fuel_type'] = data['fuel_type'].fillna('other')


petrol      163795
gasoline     74531
lpg           3950
cng            425
hybrid         171
other          159
electric        71
Name: fuel_type, dtype: int64


In [31]:
# not_repaired

print(data['not_repaired'].value_counts())

# преобладают авто, которые не восстанавливались / ремонтировались 
# однако конкретно определить все пропуски в данную категорию будет неправильно
# пропуски в столбце состовляют 20%, поэтому такую значимую часть данных определим отдельным значением other 

data['not_repaired'] = data['not_repaired'].fillna('other')


no     186886
yes     27284
Name: not_repaired, dtype: int64


**P.S.** Пропуски были заполнены одним значением - *other* и все это можно было реализовать одной строчкой в коде. Однако в некоторых случаях это было оправдано тем, что такое значение уже фигурировало в столбце. Но в случаях **gearbox** и **not_repaired** значение пропусков было заменено тем же значением (*other*), чтобы не терять часть данных, при этом не изменяя соотношение известных данных.

In [32]:
# удалим дубликаты

data = data.drop_duplicates()

<div class="alert alert-success">
<b>1 Комментарий ревьюера ✔️:</b> Здорово, что ты не забыл удалить явные дубликаты!</div>

In [33]:
# оценим влиние столбоцов на цену авто
# такие признаки, как number_of_pictures, postal_code, date_created, last_seen, registration_month, date_crawled не должны никак повлиять на цену авто
# поэтому удалим их

data = data.drop(['date_crawled','number_of_pictures','postal_code', 'date_created', 'last_seen', 'registration_month'], axis = 1)

In [34]:
# оценим статистические данные

data.describe()

,price,registration_year,power,kilometer
count,268036.000000,268036.000000,268035.000000,268035.000000
mean,4420.953995,2004.211613,110.248553,128133.359449
std,4520.051223,90.720451,196.255337,37948.946159
min,0.000000,1000.000000,0.000000,5000.000000
25%,1050.000000,1999.000000,69.000000,125000.000000
50%,2700.000000,2003.000000,105.000000,150000.000000
75%,6400.000000,2008.000000,141.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


In [35]:
# оценим количество значений 0 в столбце power  
data[data['power'] == 0].count()

price                30345
vehicle_type         30345
registration_year    30345
gearbox              30345
power                30345
model                30345
kilometer            30345
fuel_type            30345
brand                30345
not_repaired         30345
dtype: int64

In [36]:
# оценим количество значений 0 в столбце price
data[data['price'] <100].count()

price                10102
vehicle_type         10102
registration_year    10102
gearbox              10102
power                10102
model                10102
kilometer            10102
fuel_type            10102
brand                10102
not_repaired         10102
dtype: int64

In [37]:
# значения 0 в столбцах power медианными значениями для авто той марки, того же класса, той же кпп и типа топлива



data['power'] = data['power'].replace(0,np.nan)
data['power'] = data['power'].fillna(data.groupby(['brand',
                                                   'model', 
                                                   'fuel_type',
                                                   'gearbox'])['power'].transform('median'))

# значения столбца power ограничим в пределах 30 < power < 1000

data = data[(data['power']>30)&(data['power']<1000)].reset_index(drop = True)


# значения столбца registration_year ограничим в пределах 1900 < registration_year <= 2022

data = data[(data['registration_year']>1900)&(data['registration_year']<=2022)].reset_index(drop = True)

# значения столбца price оставим больше 100

data = data[(data['price']>100)].reset_index(drop = True)

In [38]:
# для последующего обучения моделей необходимо перевести текстовые значения в числовые (произвести кодировку)
# воспользуемся кодировкой Ordinal Encoding

data_encoder_col = pd.DataFrame(OrdinalEncoder().fit_transform(data[['vehicle_type',
                                                                     'gearbox',
                                                                     'model',
                                                                     'fuel_type', 
                                                                     'brand', 
                                                                     'not_repaired']]), 
                                columns = ['vehicle_type',
                                           'gearbox',
                                           'model',
                                           'fuel_type', 
                                           'brand', 
                                           'not_repaired'])

In [39]:
# заменим исходные значения в data

data[['vehicle_type', 'gearbox','model', 'fuel_type','brand','not_repaired']] = data_encoder_col[['vehicle_type',
                                                                                                  'gearbox','model',
                                                                                                  'fuel_type', 
                                                                                                  'brand',
                                                                                                  'not_repaired']].astype(int)

In [40]:
data.describe()

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,not_repaired
count,255702.000000,255702.000000,255702.000000,255702.000000,255702.000000,255702.000000,255702.000000,255702.000000,255702.000000,255702.000000
mean,4612.733045,4.153088,2003.277902,0.855210,119.248279,111.678462,128711.957669,4.736068,20.834295,0.381123
std,4523.387388,2.041078,7.179858,0.463051,53.567727,69.859391,36921.207347,1.780871,13.300311,0.656038
min,101.000000,0.000000,1910.000000,0.000000,31.000000,0.000000,5000.000000,0.000000,0.000000,0.000000
25%,1200.000000,3.000000,1999.000000,1.000000,75.000000,42.000000,125000.000000,2.000000,9.000000,0.000000
50%,2900.000000,4.000000,2003.000000,1.000000,110.000000,116.000000,150000.000000,6.000000,24.000000,0.000000
75%,6599.000000,5.000000,2008.000000,1.000000,145.000000,166.000000,150000.000000,6.000000,33.000000,1.000000
max,20000.000000,7.000000,2019.000000,2.000000,999.000000,248.000000,150000.000000,6.000000,39.000000,2.000000


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255702 entries, 0 to 255701
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   price              255702 non-null  int64  
 1   vehicle_type       255702 non-null  int64  
 2   registration_year  255702 non-null  int64  
 3   gearbox            255702 non-null  int64  
 4   power              255702 non-null  float64
 5   model              255702 non-null  int64  
 6   kilometer          255702 non-null  float64
 7   fuel_type          255702 non-null  int64  
 8   brand              255702 non-null  int64  
 9   not_repaired       255702 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 19.5 MB


По итогу предобработки данных было  исключено около ~14%~ 4.5% строк. 
Отсев строк был обоснован тем, что в исходных данных находились значения, не соответсвующие действительности (стоит отметить, что ограничения которые принимаются, также могут повлиять на эффективность моделей).

Кроме того, были удалены дубликаты, а имеющиейся пропуски заполнены значением *other*

Ввиду слабого влияния на цену ряда значений, ряд столбцов были исключены из рабочего датасета

Также для дальнейшего обучения моделей произведена кодировка текстовых значений.

## Обучение моделей

In [42]:
# создаем переменные для признаков и целевого признака

features = data.drop(columns = ['price'], axis = 1)
target = data['price']

In [43]:
# разделим наши выборки на тренировочную, тестовую в пропорции 75/25

features_train, features_test, target_train, target_test = train_test_split(features,  
                                                                            target, 
                                                                            test_size = 0.25, 
                                                                            random_state = 12345)

In [44]:
scaler = StandardScaler()
numeric = list(features.columns)
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [45]:
display(features_train.head(10))
display(features_test.head(10))

,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,not_repaired
164884,-1.053926,-1.846393,0.313096,0.087887,-0.396409,0.575867,0.710977,-0.738304,2.467167
169493,-1.053926,-4.764923,-1.843435,-1.218455,0.247272,-1.320375,0.710977,0.238420,-0.581094
65114,-2.032766,-0.317640,0.313096,0.479790,0.004103,0.575867,0.710977,-0.813437,0.943037
76509,-0.075086,0.516226,0.313096,0.386479,0.833736,0.575867,-1.534705,1.290276,-0.581094
191242,0.414334,-0.456617,0.313096,-1.293104,0.776520,0.575867,0.710977,1.290276,-0.581094
65604,-2.032766,0.238271,0.313096,-0.192044,-0.925657,0.575867,-1.534705,-0.062111,-0.581094
158930,-0.075086,0.933159,0.313096,-0.453312,-0.925657,-2.403942,0.710977,-0.062111,-0.581094
128317,1.393174,0.794181,-1.843435,2.084724,-0.754009,-0.101362,0.710977,-0.062111,-0.581094
246650,1.393174,-0.456617,0.313096,-0.770567,0.776520,0.575867,0.149556,1.290276,2.467167
170349,-0.075086,0.655203,0.313096,1.076975,1.835017,0.575867,-1.534705,-1.414498,-0.581094


,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,not_repaired
77825,-2.032766,-0.734573,0.313096,0.479790,0.004103,-0.101362,0.710977,-0.813437,-0.581094
51306,0.903754,1.350091,0.313096,0.311831,1.420201,0.575867,-1.534705,-0.437774,-0.581094
99002,0.414334,1.211114,0.313096,-0.453312,-1.483514,-1.862158,0.710977,0.313552,-0.581094
234922,-1.053926,0.516226,0.313096,0.573100,0.776520,0.575867,-1.534705,-1.564763,-0.581094
42513,0.414334,0.516226,0.313096,-0.826553,0.590568,-0.778591,0.710977,0.463817,2.467167
29211,-0.075086,-0.039685,0.313096,0.946340,-1.383386,-3.352063,0.710977,-1.414498,0.943037
115956,0.903754,-0.317640,0.313096,0.815706,0.476136,0.575867,-1.534705,-0.062111,-0.581094
224963,1.393174,-0.039685,-1.843435,0.218521,0.604872,0.575867,-1.534705,-0.813437,0.943037
252636,0.414334,0.238271,-1.843435,0.946340,-0.453625,-1.049483,0.710977,0.013022,-0.581094
162058,1.393174,-0.178662,-1.843435,-0.061409,-1.354778,0.575867,-0.411864,-0.137243,2.467167


In [46]:
# дерево решений

parameters = {'max_depth': range (1, 20, 2) }
model_dt = DecisionTreeRegressor(random_state=12345)
grid_searching = GridSearchCV(model_dt, parameters, scoring = 'neg_root_mean_squared_error', cv=5)
grid_searching.fit(features_train, target_train)
print('Лучшие параметры модели:', grid_searching.best_params_)
print('Лучшее значение RMSE:', (-1)*grid_searching.best_score_)

Лучшие параметры модели: {'max_depth': 13}
Лучшее значение RMSE: 1961.213324552283


In [47]:
# случайный лес

parameters = {'max_depth': range (1, 10, 2), 'n_estimators': range(1,50, 10) }
model_ff = RandomForestRegressor(random_state=12345, n_jobs = -1)
grid_searching = GridSearchCV(model_ff, parameters, scoring = 'neg_root_mean_squared_error', cv=5)
grid_searching.fit(features_train, target_train)
print('Лучшие параметры модели:', grid_searching.best_params_)
print('Лучшее значение RMSE:', (-1)*grid_searching.best_score_)

Лучшие параметры модели: {'max_depth': 9, 'n_estimators': 41}
Лучшее значение RMSE: 1982.6014662219725


In [48]:
# линейная регрессия

parameters = {'fit_intercept': [True, False] }
model_lr = LinearRegression(n_jobs = -1)
grid_searching = GridSearchCV(model_lr, parameters, scoring = 'neg_root_mean_squared_error', cv=5)
grid_searching.fit(features_train, target_train)
print('Лучшие параметры модели:', grid_searching.best_params_)
print('Лучшее значение RMSE:', (-1)*grid_searching.best_score_)

Лучшие параметры модели: {'fit_intercept': True}
Лучшее значение RMSE: 3031.103076845536


In [58]:
# модель градиентного бустинга

parameters = {'boosting_type': ['gbdt', 'dart', 'goss', 'rf'],
             'learning_rate': np.arange(0.1, 1.0, 0.2)}

model_lgbm = lgb.LGBMRegressor(max_depth=-1, n_jobs = -1, n_estimators = 245)
grid_searching = GridSearchCV(model_lgbm, parameters, scoring = 'neg_root_mean_squared_error', cv=5)
grid_searching.fit(features_train, target_train)
print('Лучшие параметры модели:', grid_searching.best_params_)
print('Лучшее значение RMSE:', (-1)*grid_searching.best_score_)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py", line 676, in fit
    super(LGBMRegressor, self).fit(X, y, sample_weight=sample_weight,
  File "/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py", line 538, in fit
    self._Booster = train(params, train_set,
  File "/usr/local/lib/python3

Лучшие параметры модели: {'boosting_type': 'gbdt', 'learning_rate': 0.30000000000000004}
Лучшее значение RMSE: 1634.9113517125782


С помощью GridSearch были определены гиперпараметры моделей decision_tree_regressor, random_forest_regressor, linear_regression и lgbm_regressor, при которых достигается минимальное значение RMSE 

## Анализ моделей

В проекте необходимо оценить модели по следующим критериям: 
- качество предсказания;
- скорость предсказания;
- время обучения.

Для этого определим время выполнения обучения, предсказания для моделей с гиперпараметрами, подобранными в предыдущем пункте

In [59]:
# создадим переменные для списков, в которых будут содержаться результаты

models = ['decision_tree_regressor', 'random_forest_regressor', 'linear_regression', 'lgbm_regressor']

time_fit=[]

time_predict=[]

rmse = []

In [60]:
# дерево решений

model_dt = DecisionTreeRegressor(random_state=12345, max_depth = 13)

# время обучения

start = time.time() 
model_dt.fit(features_train, target_train)
time_fit.append(time.time() - start)

# время предсказания

start = time.time() 
model_dt.predict(features_test)
time_predict.append(time.time() - start)

# результат rmse

rmse.append(mean_squared_error(target_test, model_dt.predict(features_test))**0.5)

print('Время обучения модели: ', time_fit[-1])
print('Время предсказания моедли', time_predict[-1])
print('Значение метркии rmse', rmse[-1])

Время обучения модели:  0.7458481788635254
Время предсказания моедли 0.014983654022216797
Значение метркии rmse 1922.9070869752595


In [61]:
# случайный лес

model_rf = RandomForestRegressor(random_state=12345, n_jobs = -1, max_depth = 9, n_estimators = 41)

start = time.time() 
model_rf.fit(features_train, target_train)
time_fit.append(time.time() - start)

start = time.time() 
model_rf.predict(features_test)
time_predict.append(time.time() - start)

rmse.append(mean_squared_error(target_test, model_rf.predict(features_test))**0.5)

print('Время обучения модели: ', time_fit[-1])
print('Время предсказания моедли', time_predict[-1])
print('Значение метркии rmse', rmse[-1])

Время обучения модели:  9.51130747795105
Время предсказания моедли 0.1594085693359375
Значение метркии rmse 1966.4188966789206


In [62]:
# линейная регрессия

model_lr = LinearRegression(n_jobs = -1, fit_intercept = True)

start = time.time() 
model_lr.fit(features_train, target_train)
time_fit.append(time.time() - start)

start = time.time() 
model_lr.predict(features_test)
time_predict.append(time.time() - start)

rmse.append(mean_squared_error(target_test, model_lr.predict(features_test))**0.5)

print('Время обучения модели: ', time_fit[-1])
print('Время предсказания моедли', time_predict[-1])
print('Значение метркии rmse', rmse[-1])

Время обучения модели:  0.06755614280700684
Время предсказания моедли 0.004341602325439453
Значение метркии rmse 2986.958691203651


In [64]:
# модель градиентного бустинга

model_lgbm = lgb.LGBMRegressor(max_depth=-1, 
                               n_jobs = -1,
                               boosting_type = 'gbdt', 
                               learning_rate = 0.3, 
                               n_estimators = 245)

start = time.time() 
model_lgbm.fit(features_train, target_train)
time_fit.append(time.time() - start)

start = time.time() 
model_lgbm.predict(features_test)
time_predict.append(time.time() - start)

rmse.append(mean_squared_error(target_test, model_lgbm.predict(features_test))**0.5)

print('Время обучения модели: ', time_fit[-1])
print('Время предсказания моедли', time_predict[-1])
print('Значение метркии rmse', rmse[-1])

Время обучения модели:  2.981786012649536
Время предсказания моедли 0.45664262771606445
Значение метркии rmse 1621.1491337584168


In [65]:
# представим результаты в табличном виде

pd.DataFrame({'time_fit': time_fit, 'time_predict':time_predict, 'rmse':rmse}, index = models).style.highlight_min(color='coral')

,time_fit,time_predict,rmse
decision_tree_regressor,0.745848,0.014984,1922.907087
random_forest_regressor,9.511307,0.159409,1966.418897
linear_regression,0.067556,0.004342,2986.958691
lgbm_regressor,2.981786,0.456643,1621.149134


Из таблицы видно, что быстрее всего с обучением модели и предсказанием результатов справляется линейная регрессия, но при этом она хуже всего предсказывает результат (метрика rmse - наибольшая)

Модель градиентного бустинга имеем наилучшую метрику rmse, но при этом проигрывает остальным моделям в предсказании и достаточно долго обучается.

Наиболее оптимальной с точки зрения затрат времени на обучения модели, предсказания значений и метрикой rmse является модель Дерева решений.

Стоит отметить, что на результаты могут повлиять значения гиперпараметров параметров, разделение данных на тренировочную и тестовую выборки. Также определенное влияние оказывает первичная предобработка датасета.

In [66]:
# сравним результаты с наивной моделью

dummy_model = DummyRegressor(strategy='mean')

dummy_model.fit(features_train, target_train)

print('rmse наивной модели:', mean_squared_error(target_test, dummy_model.predict(features_test))**0.5)

rmse наивной модели: 4511.507943470774


Значение метрики rmse много хуже значения соответствующей метрики линейной регрессии (худший результат в исследовании), что говорит об адекватности полученных ранее результатов